In [1]:
from keras.datasets import mnist
from keras.utils import np_utils
from keras.layers import Input, Dense, Dropout, Activation, Flatten, Convolution2D, ZeroPadding2D, MaxPooling2D
from keras.models import Model
from keras.layers import concatenate

Using Theano backend.
Using cuDNN version 5103 on context None
Mapped name None to device cuda: Graphics Device (0000:07:00.0)


In [2]:
batch_size = 128
nb_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

# Load

In [3]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
input_shape = (1, img_rows, img_cols)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


('X_train shape:', (60000, 1, 28, 28))
(60000, 'train samples')
(10000, 'test samples')


# Design

In [4]:
input_layer = Input(shape=(input_shape))

# Network thread 1 
layers = Convolution2D(nb_filters, kernel_size, activation='relu')(input_layer)
layers = Convolution2D(nb_filters, kernel_size, activation='relu')(layers)
layers = MaxPooling2D(pool_size)(layers)
layers = Dropout(0.25)(layers)
layers = Flatten()(layers)

# Network thread 2
layers1 = Convolution2D(nb_filters, (2,2), activation='relu')(input_layer)
layers1 = Convolution2D(nb_filters, (2,2), activation='relu')(layers1)
layers1 = MaxPooling2D(pool_size)(layers1)
layers1 = Dropout(0.25)(layers1)
layers1 = Flatten()(layers1)

# Network thread 3
layers2 = Convolution2D(nb_filters, (1,1), activation='relu')(input_layer)
layers2 = Convolution2D(nb_filters, (1,1), activation='relu')(layers2)
layers2 = MaxPooling2D(pool_size)(layers2)
layers2 = Dropout(0.25)(layers2)
layers2 = Flatten()(layers2)

# Combine (just concatenate here) three network threads
layers = concatenate([layers, layers1, layers2])
layers = Dense(128, activation='relu')(layers)
layers = Dropout(0.5)(layers)
output_layer = Dense(nb_classes, activation='softmax')(layers)

# Final Model Construction
model = Model(inputs=input_layer, outputs=output_layer)

# Train

In [5]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,
          verbose=1, validation_data=(X_test, Y_test))


Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 6s - loss: 0.3383 - acc: 0.8971 - val_loss: 0.0849 - val_acc: 0.9742
Epoch 2/12
60000/60000 [==============================] - 6s - loss: 0.1247 - acc: 0.9628 - val_loss: 0.0568 - val_acc: 0.9815
Epoch 3/12
60000/60000 [==============================] - 6s - loss: 0.0945 - acc: 0.9724 - val_loss: 0.0464 - val_acc: 0.9850
Epoch 4/12
60000/60000 [==============================] - 6s - loss: 0.0783 - acc: 0.9773 - val_loss: 0.0425 - val_acc: 0.9857
Epoch 5/12
60000/60000 [==============================] - 6s - loss: 0.0668 - acc: 0.9807 - val_loss: 0.0371 - val_acc: 0.9874
Epoch 6/12
60000/60000 [==============================] - 6s - loss: 0.0591 - acc: 0.9828 - val_loss: 0.0347 - val_acc: 0.9887
Epoch 7/12
60000/60000 [==============================] - 6s - loss: 0.0558 - acc: 0.9835 - val_loss: 0.0328 - val_acc: 0.9888
Epoch 8/12
60000/60000 [==============================] - 6s 

# Test

In [6]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

('Test loss:', 0.031789702635592401)
('Test accuracy:', 0.98929999999999996)
